In [1]:
from PIL import Image
import os
import numpy as np
import cv2

In [2]:
def align_image(reference_dir, target_dir):
    sift = cv2.SIFT_create()
    reference_img = cv2.imread(reference_dir)
    target_img = cv2.imread(target_dir)

    keypointsRef, descriptorsRef = sift.detectAndCompute(reference_img, None)
    keypointsTarget, descriptorsTarget = sift.detectAndCompute(target_img,None)
    if keypointsRef is None or keypointsTarget is None:
        raise ValueError("Failed to detect keypoints.")
    if descriptorsRef is None or descriptorsTarget is None:
        raise ValueError("Failed to compute descriptors.")
    matcher = cv2.FlannBasedMatcher({"algorithm": 1, "trees": 5}, {"checks": 50})
    matches = matcher.knnMatch(descriptorsRef, descriptorsTarget, k = 2)

    good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]

    pointsRef = np.float32([keypointsRef[m.queryIdx].pt for m in good_matches]).reshape(-1,1,2)
    pointsTarget = np.float32([keypointsTarget[m.trainIdx].pt for m in good_matches]).reshape(-1,1,2)

    homography, _ = cv2.findHomography(pointsTarget, pointsRef, cv2.RANSAC)
    
    aligned_img = cv2.warpPerspective(target_img, homography, (2560,2560))

    cv2.imwrite(target_dir, aligned_img)

def align_dir(reference_dir, target_dir):
    ref_list = os.listdir(reference_dir)
    tar_list = os.listdir(target_dir)
    for i in range(len(os.listdir(reference_dir))):
        ref_path = os.path.join(reference_dir, ref_list[i])
        tar_path = os.path.join(target_dir, tar_list[i])
        try:
            align_image(ref_path, tar_path)
        except Exception as e:
            print(f"Failed to align {ref_list[i]} and {tar_list[i]}: {e}")

        print(f"Aligning: {i+1}/{len(ref_list)}\t\t\t\t\t\t\t\t", end="\r")

def split_image(image, image_name, output_dir, grid_size):
    # Open the image file
    image_width, image_height = image.size

    # Calculate the size of each tile
    tile_width = image_width // grid_size[0]
    tile_height = image_height // grid_size[1]

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Loop through the grid and save smaller images
    for row in range(grid_size[1]):
        for col in range(grid_size[0]):
            # Calculate the coordinates of the tile
            left = col * tile_width
            upper = row * tile_height
            right = left + tile_width
            lower = upper + tile_height

            # Crop the tile from the image
            cropped_image = image.crop((left, upper, right, lower))

            # Save the cropped image
            tile_filename = f"{image_name[:-4]}_tile_{row}_{col}.jpg"
            cropped_image.save(os.path.join(output_dir, tile_filename))

    # print(f"Image successfully split into {grid_size[0]}x{grid_size[1]} grid and saved in {output_dir}.")

def split_dir(raw_dir, process_dir, grid):
    raw_files = os.listdir(raw_dir)
    for i in range(len(raw_files)):
        raw_path = os.path.join(raw_dir, raw_files[i])
        try:
            with Image.open(raw_path) as img:
                split_image(img, raw_files[i], process_dir, grid)
                print(f"Splitting: {i+1}/{len(raw_files)} images into {grid[0]}x{grid[1]} grid and saved to {process_dir}\t\t\t\t\t\t\t\t", end="\r")

        except Exception as e:
            print(f"Failed to split {raw_files[i]}: {e}")
        

def resize_dir(raw_dir, dst_dir, target_size):
    raw_files = os.listdir(raw_dir)
    for i in range(len(raw_files)):
        raw_path = os.path.join(raw_dir, raw_files[i])
        dst_path = os.path.join(dst_dir, raw_files[i])
        if not os.path.isfile(raw_path):
            continue
        try:
            with Image.open(raw_path) as img:
                img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
                img_resized.save(dst_path)
                print(f"Resizing: {i+1}/{len(raw_files)} images saved to {dst_dir}\t\t\t\t\t\t\t\t", end="\r")

        except Exception as e:
            print(f"Failed to process {raw_files[i]}: {e}, {target_size}")


def gradient(value, range, max_val):
    normalized = (value - range[0]) / (range[1] - range[0])
    normalized = max(0, min(1, normalized))  # Clamp to [0, 1]
    # Map the normalized value to an RGB scale
    if normalized < 0.5: 
        r = 255
        g = int(255 * normalized * 2)
        b = 0 
    elif normalized <= 1: 
        r = 255 - int(255 * (normalized - 0.5) * 2)
        g = 255  # Scale to [0, 255]
        b = 0
    # else: 
    #     r = int(255 * (normalized - 0.66) * 3)  # Scale to [0, 255]
    #     g = 255 - r
    #     b = 0

    return [r, g, b]
def mono(val, range, new_range):
    converted_value = (val + 1) * 127.5
    return int(round(converted_value))

def get_timestamps(dir,  delimiter = "%,", timestamps_pos = -1, extension = ".jpg"):
    timestamps = set()
    ts=[]
    for filename in os.listdir(dir):
        if filename.__contains__(extension):
            timestamp = filename.split(delimiter)[timestamps_pos].split(extension)[0]
            timestamps.add(timestamp)
            ts.append(timestamp)
    return timestamps, ts

def create_ndvi_image(red_path, noir_path, ndvi_path, stamp):
    red = Image.open(red_path)
    noir = Image.open(noir_path).resize(red.size)

    red_pix = np.array(red)
    noir_pix = np.array(noir)

    mono_ndvi_pix = np.zeros(red_pix.shape)

    for x in range(len(noir_pix)):
        for y in range(len(noir_pix[x])):
            nir = int(np.average(noir_pix[x][y]))
            red = int(np.average(red_pix[x][y]))
            ndvi_value = (nir - red) / (nir + red)
            mono_ndvi_pix[x][y] = mono(ndvi_value, [-1,1], [0,255])
    mono_ndvi = Image.fromarray(mono_ndvi_pix.astype(np.uint8))
    mono_ndvi.save(ndvi_path + "mono_ndvi" + str(stamp) + ".jpg")

def generate_ndvi_dir(red_dir, noir_dir, ndvi_dir):
    red_list = os.listdir(red_dir)
    noir_list = os.listdir(noir_dir)
    timestamps = get_timestamps(red_dir)[1]
    for i in range(len(red_list)):
        red_path = os.path.join(red_dir,red_list[i])
        noir_path = os.path.join(noir_dir, noir_list[i])
        stamp = timestamps[i]
        create_ndvi_image(red_path, noir_path, ndvi_dir, stamp)
        print(f"Generating NDVI Map: {i+1}/{len(red_list)} images saved to {ndvi_dir}\t\t\t\t\t\t\t\t", end="\r")


def clean_nonpairs(rgb_dir, noir_dir, delimiter = "%,", timestamps_pos = -1, extension = ".jpg"):
    timestampsRGB = get_timestamps(rgb_dir, delimiter=delimiter, timestamps_pos=timestamps_pos, extension=extension)[0]
    timestampsNOIR = get_timestamps(noir_dir, delimiter=delimiter, timestamps_pos=timestamps_pos, extension=extension)[0]

    common_timestamps = timestampsNOIR.intersection(timestampsRGB)

    for filename in os.listdir(rgb_dir):
        if filename.endswith(extension):
            timestamp = filename.split(delimiter)[timestamps_pos].split(extension)[0]
            if timestamp not in common_timestamps:
                os.remove(os.path.join(rgb_dir, filename))
                print(f"Removed {filename} from {rgb_dir}")

    for filename in os.listdir(noir_dir):
        if filename.endswith(extension):
            timestamp = filename.split(delimiter)[timestamps_pos].split(extension)[0]
            if timestamp not in common_timestamps:
                os.remove(os.path.join(noir_dir, filename))
                print(f"Removed {filename} from {noir_dir}")

def make_ir_mono(src_dir, dst_dir):
    files = os.listdir(src_dir)
    for i in range(len(files)):
        raw_path = os.path.join(src_dir, files[i])
        dst_path = os.path.join(dst_dir, files[i])
        try:
            with Image.open(raw_path) as img:
                mono = img.convert('L')
                mono.save(dst_path)
        except Exception as e:
            print(f"{raw_path} ---> {dst_path}")
            print(f"Failed to apply IR-mono to {files[i]}: {e}")
        
        print(f"Making IR Mono: {i+1}/{len(files)} images saved to {dst_dir}\t\t\t\t\t\t\t\t", end="\r")



def make_red_mono(src_dir, dst_dir):
    files = os.listdir(src_dir)
    for i in range(len(files)):
        raw_path = os.path.join(src_dir, files[i])
        dst_path = os.path.join(dst_dir, files[i])
        try:
            with Image.open(raw_path) as img:
                rgb = img.convert('RGB')
                red, blue, green = rgb.split()
                red.save(dst_path)
        except Exception as e:
            print(f"Failed to apply red-mono to {files[i]}: {e}")
        print(f"Making RED Mono: {i+1}/{len(files)} images saved to {dst_dir}\t\t\t\t\t\t\t\t", end="\r")


In [ ]:
def stitch_image(image_name, input_dir, output_path, grid_size):
    # Get tile dimensions from any tile in the directory
    sample_tile = Image.open(os.path.join(input_dir, f"{image_name[:-4]}_tile_0_0.jpg"))
    tile_width, tile_height = sample_tile.size

    # Create a blank canvas for the final image
    stitched_image = Image.new("RGB", (tile_width * grid_size[0], tile_height * grid_size[1]))

    # Loop through tiles and paste them onto the blank canvas
    for row in range(grid_size[1]):
        for col in range(grid_size[0]):
            tile_filename = f"{image_name[:-4]}_tile_{row}_{col}.jpg"
            tile_path = os.path.join(input_dir, tile_filename)

            if os.path.exists(tile_path):  # Ensure the tile exists
                tile = Image.open(tile_path)
                stitched_image.paste(tile, (col * tile_width, row * tile_height))

    # Save the reassembled image
    stitched_image.save(output_path)
    print(f"Stitched image saved as {output_path}")

# Example Usage:
stitch_image("image.jpg", "output_tiles", "reconstructed_image.jpg", (4, 4))

In [3]:
clean_nonpairs("_raw/noir/", "_raw/rgb/")
make_ir_mono("_raw/noir/", "_raw/noir/")
resize_dir("_raw/rgb/", "rgb/", (2560,2560))
resize_dir("_raw/noir/", "nir_mono/",(2560,2560))
align_dir("nir_mono/", "rgb/")
make_red_mono("rgb/", "red_mono/")
split_dir("rgb/", "split_dataset/RGB/", [10,10])
split_dir("red_mono/", "split_dataset/RED_MONO/",[10,10])
split_dir("nir_mono/", "split_dataset/NIR_MONO/",[10,10])
generate_ndvi_dir("split_dataset/RED_MONO/", "split_dataset/NIR_MONO/", "split_dataset/NDVI_MONO/") # This takes WAY too much memory to process, so make sure to split RGB, NIR, and RED MONO before this step

Removed NoIR_Camera2025%,18_153029.jpg from _raw/noir/
Removed NoIR_Camera2025%,18_153334.jpg from _raw/noir/
Removed NoIR_Camera2025%,18_153430.jpg from _raw/noir/
Removed NoIR_Camera2025%,18_153441.jpg from _raw/noir/
Removed NoIR_Camera2025%,18_153955.jpg from _raw/noir/
Removed NoIR_Camera2025%,18_154548.jpg from _raw/noir/
Removed NoIR_Camera2025%,18_155417.jpg from _raw/noir/
Removed NoIR_Camera2025%,18_155608.jpg from _raw/noir/
Removed NoIR_Camera2025%,18_155744.jpg from _raw/noir/
Removed NoIR_Camera2025%,18_155944.jpg from _raw/noir/
Removed RGB_Camera2025%,18_153443.jpg from _raw/rgb/
Removed RGB_Camera2025%,18_153616.jpg from _raw/rgb/
Removed RGB_Camera2025%,18_153640.jpg from _raw/rgb/
Removed RGB_Camera2025%,18_154128.jpg from _raw/rgb/
Removed RGB_Camera2025%,18_154202.jpg from _raw/rgb/
Removed RGB_Camera2025%,18_154320.jpg from _raw/rgb/
Removed RGB_Camera2025%,18_154817.jpg from _raw/rgb/
Removed RGB_Camera2025%,18_154835.jpg from _raw/rgb/
Removed RGB_Camera2025%,18